In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib
matplotlib.rcParams['figure.facecolor'] = 'white'
matplotlib.rcParams['savefig.dpi'] = 300

import sys
import json
import os
plt.style.use('../styles/general.mplstyle')

In [2]:
def show_fig(datas, 
             name = None, 
             *,
             dt = 5e-5,
             nrows = 4, 
             ncols = 5,
             color_reference="0.2",
             ls_reference="-",
             lss = None,
             colors = None,
             xlim = None, 
             ylim = None,
             n_sections = 20,
             all_len = 100000,
             start = -80, 
             step = 5):
    
    if colors is None:
        colors = [color_reference] + [f'C{i}' for i in range(len(datas) - 1)]
    else:
        colors = [color_reference] + colors
    
    if lss is None:
        lss = [ls_reference] + ['-' for i in range(len(datas) -1)]
    else:
        lss = [ls_reference] + lss
    
    
    fig, axes = plt.subplots(nrows=nrows, 
                             ncols=ncols,
                             figsize=plt.figaspect(nrows / ncols) * 2,
                             sharex=True)
    fig.suptitle(name)
    
    len_trace = int(all_len/n_sections)
    split_indices = np.linspace(0, all_len, n_sections + 1).astype(int)
    
    t = np.arange(len_trace) * dt
    
    if xlim is None:
        xlim = (0.,len_trace * dt )
 
    i_global = 0
    
    for i_row in range(nrows):
        for i_col in range(ncols):
            ax = axes[i_row, i_col]
            plt.sca(ax)
            i_start, i_end = split_indices[i_global], split_indices[i_global + 1]
            slice_current = slice(i_start, i_end)
            
            for k, data in enumerate(datas):
                data_show = data[slice_current]
                plt.plot(t, data_show, color=colors[k], ls = lss[k])
                if ylim is None:
                    plt.ylim(min(data_show)-100, max(data_show)+50)
                else:
                    plt.ylim(ylim)
            
            step_value = start + step * i_global
            title = f'step {step_value} mV'
            plt.title(title)
            plt.xlim(xlim)
            i_global += 1

            

In [3]:
import pypoptim

In [ ]:
sys.path.append('../pypoptim/mpi_scripts/')

from loss_utils import calculate_loss
from io_utils import collect_results

In [ ]:
from pypoptim.losses import RMSE

In [ ]:
sys.path.append('pypoptim/mpi_scripts/')
from ina_model import InaModel

dirname = '../src/model_ctypes/M1/'
filename_so = os.path.join(dirname, 'ina.so')


filename_so_abs = os.path.abspath(filename_so)
INA = InaModel(filename_so_abs)

In [ ]:
legend_constants = pd.read_csv(os.path.join(dirname, "legend_test.csv"), index_col='name')['value'] 
legend_states = pd.read_csv(os.path.join(dirname, "legend_states.csv"), index_col='name')['value'] 
legend_algebraic = pd.read_csv(os.path.join(dirname, "legend_algebraic.csv"), index_col='name')['value'] 

In [ ]:
S = legend_states.copy()
C = legend_constants.copy()
A = legend_algebraic.copy()

In [ ]:
df_protocol_act = pd.read_csv('../data/protocols/protocol_trace.csv')
df_protocol_inact = pd.read_csv('../data/protocols/protocol_inact.csv')
df_initial_state_protocol = pd.read_csv('../data/protocols/protocol_initial_state.csv')

In [ ]:
n_sections_act = 20
n_sections_inact = 15

In [ ]:
filename = '../data/BigTableM1.csv'

BigTableM1 = pd.read_csv(
    filename,
    index_col=[0, 1],
#     names=["data", "case"],
)
BigTableM1 = BigTableM1.T
BigTableM1

In [ ]:
param_example = BigTableM1['2020_12_19_0039']['211124_002757']

In [ ]:
act_states, act_algebraic = INA.run(A,
                                    S,
                                    param_example,# param for change
                                    df_protocol_act,
                                    df_initial_state_protocol,
                                    n_sections=n_sections_act,
                                    return_algebraic=True)

In [ ]:
act_states

In [ ]:
act_algebraic

In [ ]:
example_activation = pd.read_csv('../data/real/activation/2020_12_19_0039.csv')

In [ ]:
what_you_want_to_show_activation = [
                                    example_activation.I_out, 
                                    act_states.I_out, 
                                    act_algebraic.I_Na
                                    ]


show_fig(    
    what_you_want_to_show_activation,
    
    ###   how to show graph  ###
    # name for graph
    name = 'Activation',
    
    # color for every data in what_you_want_to_show
#     colors=['C0','C1','C0', 'C1'],
    
    # linestyle for every data in what_you_want_to_show
#     lss =['-','-.', ':', '-.'],
    
    # where show every step
    xlim=(0.1,0.15),
    ylim=(-500, 150),
        )

# Inactivation

In [ ]:
example_inactivation = pd.read_csv('../data/real/inactivation/2020_12_19_0036.csv')

In [ ]:
inact_states, inact_algebraic = INA.run(A,
                                        S,
                                        param_example,# param for change
                                        df_protocol_inact,
                                        df_initial_state_protocol,
                                        n_sections=n_sections_inact,
                                        return_algebraic=True)


In [ ]:
start_inact = -140
step_inact = 10
len_inact = 180000
nrows_inact = 5
ncols_inact = 3

In [ ]:
what_you_want_to_show = [example_inactivation, inact_states.I_out, inact_algebraic.I_Na]


show_fig(    
    what_you_want_to_show,
    
    ###   how to show graph  ###
    # name for graph
    name = 'inact',
    
    # color for every data in what_you_want_to_show
#     colors=['C0','C1','C0', 'C1'],
    
    # linestyle for every data in what_you_want_to_show
#     lss =['-','-.', ':', '-.'],
    
    # where show every step
    xlim=(0.,0.6),
    ylim=(-500, 150),
    

    ### Parameters for inactivation ### 
    ### Don't change ###
    start = start_inact,
    step = step_inact,
    n_sections=n_sections_inact,
    nrows=nrows_inact,
    ncols=ncols_inact,
    all_len=len_inact,
        )

In [1]:
pd.DataFrames(m2['sols'])

NameError: name 'pd' is not defined

In [ ]:
 res = solve_ivp(
        model.calculate_rates,
        t_span=args.solver_options.t_span,
        y0=y0,
        method=args.solver_options.method,
        max_step=args.solver_options.max_step,
        args=[parameters, algebraics, states_keys],
    )